[View in Colaboratory](https://colab.research.google.com/github/jasonchanhku/LightGBM/blob/master/notebooks/LightBGM_GC_Practice.ipynb)

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import lightgbm as lgb
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score


In [10]:
#data preprocessing
dataset = pd.read_csv('https://raw.githubusercontent.com/jasonchanhku/LightGBM/master/data/Social_Network_Ads.csv')
dataset.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [0]:
# make array of datasets
X = dataset.iloc[:, [2, 3]].values
y = dataset.iloc[:, 4].values

In [0]:
# train test split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [14]:
# Feature scaling
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [46]:
# Converting into LGB dataset type
d_train = lgb.Dataset(x_train, label = y_train)
d_valid = lgb.Dataset(x_test, label = y_test )
params = {}
params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.5
params['num_leaves'] = 10
params['min_data'] = 50
params['max_depth'] = 10

clf = lgb.train(params, d_train, 500, verbose_eval=100, valid_sets=[d_train, d_valid], valid_names=['Train','Valid'], early_stopping_rounds=10)

Training until validation scores don't improve for 10 rounds.
[100]	Train's binary_logloss: 0.591596	Valid's binary_logloss: 0.571428
[200]	Train's binary_logloss: 0.519917	Valid's binary_logloss: 0.488471
[300]	Train's binary_logloss: 0.467314	Valid's binary_logloss: 0.428397
[400]	Train's binary_logloss: 0.427317	Valid's binary_logloss: 0.384043
[500]	Train's binary_logloss: 0.39653	Valid's binary_logloss: 0.349531
Did not meet early stopping. Best iteration is:
[500]	Train's binary_logloss: 0.39653	Valid's binary_logloss: 0.349531


In [0]:
# Prediction
y_pred = clf.predict(x_test)


In [0]:
# Convert to binary

for i in range(0, 100):
  if y_pred[i] > 0.5:
    y_pred[i] = 1
  else:
    y_pred[i] = 0

In [49]:
y_pred

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0.,
       0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1.,
       0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1.,
       0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1.])

In [50]:
# Confusion Matrix and Accuracy
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[64  4]
 [ 4 28]]


In [51]:
acc = accuracy_score(y_pred, y_test)
print(acc)

0.92
